It is recommended to use google collab for work.
Importing libraries.

In [1]:
!pip install pycaret
!pip install transformers
from transformers import BertTokenizer, BertModel, RobertaConfig, RobertaModel, RobertaTokenizer
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from pycaret.classification import *
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.4/484.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 19.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

Checking that GPUs are available.

In [2]:
torch.cuda.is_available()

True

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

Uploading data

In [4]:
data = pd.read_csv('GPT_vs_H.csv')

In [5]:
df = data[["ans_text", 'label']]

In [6]:
df.head()

,ans_text,label
0,"Да ничего особенного. :)\nКорпус должен быть, ...",hu_answer
1,Здравствуйте! Спасибо за интересный вопрос. Дл...,ai_answer
2,"To load a reusable UITableViewCell from a Nib,...",ai_answer
3,"Actually, since you are building the cell in I...",hu_answer
4,You can use -[UIButton setTitleColor:forState:...,hu_answer


Replacing our target variable with 0 and 1

In [7]:
df['label'] = df.label.replace(["ai_answer", "hu_answer"], [0,1])
df.head()

<ipython-input-7-83134e8c4f53>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df.label.replace(["ai_answer", "hu_answer"], [0,1])


,ans_text,label
0,"Да ничего особенного. :)\nКорпус должен быть, ...",1
1,Здравствуйте! Спасибо за интересный вопрос. Дл...,0
2,"To load a reusable UITableViewCell from a Nib,...",0
3,"Actually, since you are building the cell in I...",1
4,You can use -[UIButton setTitleColor:forState:...,1


We load 3 pre-trained models and tokenizers.

In [10]:
tokenizer_ru = RobertaTokenizer.from_pretrained('sberbank-ai/ruRoberta-large')
model_ru = BertModel.from_pretrained("sberbank-ai/ruRoberta-large")

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some weights of the model checkpoint at sberbank-ai/ruRoberta-large were not used when initializing BertModel: ['roberta.encoder.layer.21.attention.self.query.weight', 'roberta.encoder.layer.17.attention.output.dense.bias', 'roberta.encoder.layer.22.intermediate.dense.bias', 'roberta.encoder.layer.3.attention.self.query.weight', 'roberta.encoder.layer.15.output.LayerNorm.bias', 'roberta.encoder.layer.4.attention.self.value.bias', 'roberta.encoder.layer.2.attention.self.value.bias', 'roberta.encoder.layer.7.attention.output.dense.weight', 'roberta.encoder.layer.10.attention.self.query.bias', 'roberta.encoder.layer.4.attention.output.dense.weight', 'roberta.encoder.layer.5.attention.self.query.bias', 'roberta.encoder.layer.19.attention.self.key.bias', 'roberta.encoder.layer.0.attention.self.value.bias', 'lm_head.decoder.bias', 'roberta.encoder.layer.14.attention.self.key.weight', 'roberta.encoder.layer.17.output.LayerNorm.weight', 'roberta.encoder.layer.12.intermediate.dense.weight', 'ro

In [11]:
tokenizer_eng = RobertaTokenizer.from_pretrained('Roberta-large')
model_eng = BertModel.from_pretrained("Roberta-large")

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some weights of the model checkpoint at Roberta-large were not used when initializing BertModel: ['roberta.encoder.layer.21.attention.self.query.weight', 'roberta.encoder.layer.17.attention.output.dense.bias', 'roberta.encoder.layer.22.intermediate.dense.bias', 'roberta.encoder.layer.3.attention.self.query.weight', 'roberta.encoder.layer.4.attention.self.value.bias', 'roberta.encoder.layer.15.output.LayerNorm.bias', 'roberta.encoder.layer.2.attention.self.value.bias', 'roberta.encoder.layer.7.attention.output.dense.weight', 'roberta.encoder.layer.10.attention.self.query.bias', 'roberta.encoder.layer.4.attention.output.dense.weight', 'roberta.encoder.layer.5.attention.self.query.bias', 'roberta.encoder.layer.19.attention.self.key.bias', 'roberta.encoder.layer.0.attention.self.value.bias', 'roberta.encoder.layer.14.attention.self.key.weight', 'roberta.encoder.layer.17.output.LayerNorm.weight', 'roberta.encoder.layer.12.intermediate.dense.weight', 'roberta.encoder.layer.23.attention.self.

In [12]:
tokenizer_mult = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model_mult = BertModel.from_pretrained("bert-base-multilingual-cased")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


We create 3 DataFrames from embedding on 3 models and combine them into 1 DataFrame with 1500 features.

In [13]:
fin_df = pd.DataFrame()
# Load the tokenizer and model
for leng in [[tokenizer_mult, model_mult, 'mult'],[tokenizer_ru, model_ru, 'ru'],[tokenizer_eng, model_eng, 'eng']]:
  tokenizer, model, suffix = leng
  model = model.to(device)

  # Function to get BERT embeddings for a given text
  def get_bert_embeddings(text):
      encoded_input = tokenizer(text, return_tensors='pt', max_length=500, truncation=True, padding=True)
      encoded_input.to(device)
      with torch.no_grad():
          outputs = model(**encoded_input)
          embeddings = outputs.last_hidden_state[:, 0, :].squeeze()  # Extract the embeddings from the last layer
      return embeddings.tolist()

  # Create a new DataFrame to store the embedding features and labels
  new_columns = [f'embedding_{suffix}_{i+1}' for i in range(500)]
  new_df = pd.DataFrame(columns=new_columns)

  # Iterate over each row in the original DataFrame
  for index, row in tqdm(df.iterrows()):
      text = row['ans_text']
      # label = row['label']

      # Get the BERT embeddings for the text
      embeddings = get_bert_embeddings(text)

      # Ensure that the embeddings list has exactly 100 elements
      embeddings = embeddings[:500] + [0] * max(0, 500 - len(embeddings))

      # Create a new row for the embedding features and label
      new_row = embeddings #+ [int(label)]

      # Add the new row to the new DataFrame
      new_df.loc[index] = new_row
  fin_df = pd.concat([fin_df,new_df], axis=1)


4819it [01:53, 42.49it/s]
4819it [05:24, 14.84it/s]
4819it [06:47, 11.84it/s]


In [ ]:
fin_df.head()

,embedding_mult_1,embedding_mult_2,embedding_mult_3,embedding_mult_4,embedding_mult_5,embedding_mult_6,embedding_mult_7,embedding_mult_8,embedding_mult_9,embedding_mult_10,...,embedding_eng_491,embedding_eng_492,embedding_eng_493,embedding_eng_494,embedding_eng_495,embedding_eng_496,embedding_eng_497,embedding_eng_498,embedding_eng_499,embedding_eng_500
0,0.030713,-0.103091,0.040415,0.278039,0.006219,0.316357,-0.120499,0.100733,0.116304,0.132695,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.335275,-0.161907,-0.113344,0.249395,0.046125,0.581767,-0.127561,-0.058241,0.004266,0.069076,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000501,-0.181456,-0.216668,0.393028,0.049741,0.215898,-0.049870,0.187981,0.096129,-0.117780,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.025947,-0.018504,-0.001142,0.128156,0.069750,0.130710,-0.226540,-0.009126,-0.051182,0.173942,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.367427,-0.226657,-0.107649,0.096227,-0.084828,-0.099241,0.172423,0.020631,0.027104,-0.158381,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Add target variable in our DataFrame

In [19]:
fin_df['label'] = df['label']
fin_df.head()

,embedding_mult_1,embedding_mult_2,embedding_mult_3,embedding_mult_4,embedding_mult_5,embedding_mult_6,embedding_mult_7,embedding_mult_8,embedding_mult_9,embedding_mult_10,...,embedding_eng_492,embedding_eng_493,embedding_eng_494,embedding_eng_495,embedding_eng_496,embedding_eng_497,embedding_eng_498,embedding_eng_499,embedding_eng_500,label
0,0.030713,-0.103091,0.040415,0.278039,0.006219,0.316357,-0.120499,0.100733,0.116304,0.132695,...,0.904381,0.755190,0.941344,0.490967,-0.758182,-0.806274,0.141161,2.167080,-0.757247,1
1,-0.335275,-0.161907,-0.113344,0.249395,0.046125,0.581767,-0.127561,-0.058241,0.004266,0.069076,...,0.854790,0.775398,0.956989,0.499140,-0.793900,-0.837113,0.177271,2.188793,-0.790205,0
2,0.000501,-0.181456,-0.216668,0.393028,0.049741,0.215898,-0.049870,0.187981,0.096129,-0.117780,...,0.316524,0.864837,1.226036,0.233724,-0.897735,-0.713893,0.366637,1.811545,-0.362900,0
3,0.025947,-0.018504,-0.001142,0.128156,0.069750,0.130710,-0.226540,-0.009126,-0.051182,0.173942,...,0.635283,0.733402,1.492425,-0.086886,-0.786468,-0.479859,0.390750,1.963224,-0.429552,1
4,0.367427,-0.226657,-0.107649,0.096227,-0.084828,-0.099241,0.172423,0.020631,0.027104,-0.158381,...,0.742766,0.946771,1.019115,0.173046,-0.962292,-0.973724,0.266787,1.530807,-0.477865,1


We divide our DataFrame into an explane and predict variable

In [20]:
y = np.array(fin_df.label)
X = fin_df.drop(columns=['label'])
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)

In [21]:
setup(X,target=y ,session_id=42,train_size=0.8)

,Description,Value
0,Session id,42
1,Target,target
2,Target type,Binary
3,Original data shape,"(4819, 1501)"
4,Transformed data shape,"(4819, 1501)"
5,Transformed train set shape,"(3855, 1501)"
6,Transformed test set shape,"(964, 1501)"
7,Numeric features,1500
8,Preprocess,True
9,Imputation type,simple


Checking the results of classification models

In [22]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9575,0.9913,0.9535,0.9607,0.9570,0.9149,0.9151,3.9150
ridge,Ridge Classifier,0.9572,0.0000,0.9525,0.9611,0.9567,0.9144,0.9146,0.7750
lda,Linear Discriminant Analysis,0.9300,0.9785,0.9201,0.9381,0.9288,0.8599,0.8604,4.7360
xgboost,Extreme Gradient Boosting,0.9300,0.9827,0.9165,0.9411,0.9285,0.8599,0.8605,70.8420
lightgbm,Light Gradient Boosting Machine,0.9300,0.9812,0.9180,0.9398,0.9285,0.8599,0.8606,21.9380
gbc,Gradient Boosting Classifier,0.9214,0.9765,0.9133,0.9277,0.9202,0.8428,0.8433,137.6750
rf,Random Forest Classifier,0.9045,0.9640,0.8731,0.9308,0.9007,0.8090,0.8110,9.2250
knn,K Neighbors Classifier,0.9017,0.9523,0.8956,0.9057,0.9005,0.8033,0.8037,0.5230
et,Extra Trees Classifier,0.8991,0.9660,0.8585,0.9331,0.8941,0.7981,0.8008,2.1380
ada,Ada Boost Classifier,0.8955,0.9575,0.8883,0.9007,0.8940,0.7909,0.7916,26.6750


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Best Model - LogisticRegression
Look more detail

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)

In [25]:
for i in ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']:
  log = LogisticRegression(solver=i)
  log.fit(X_train, y_train)
  pred = log.predict(X_test)
  print(i,"F1", f1_score(y_test, pred))

lbfgs F1 0.9480737018425461
liblinear F1 0.9472803347280335
newton-cg F1 0.9472803347280335
newton-cholesky F1 0.9464882943143812
sag F1 0.9455155071248952
saga F1 0.9405857740585775


In [29]:
for i in [0.01, 0.1, 1, 5, 20, 100]:
  log = LogisticRegression(solver='lbfgs', C=i)
  log.fit(X_train, y_train)
  pred = log.predict(X_test)
  print(i,"F1", f1_score(y_test, pred))

0.01 F1 0.9093959731543625
0.1 F1 0.9360269360269361
1 F1 0.9480737018425461
5 F1 0.9438390611902765
20 F1 0.9361344537815125
100 F1 0.9367088607594937
